In [ ]:
from torchvision import transforms as T
preprocess = T.Compose([
    T.ToTensor(),
    T.Normalize(mean=[0.485, 0.456, 0.406],
                std=[0.229, 0.224, 0.225]),
    T.Lambda(lambda x: x.mul_(255))
])

postprocess=T.Compose([
    T.Lambda(lambda x: x.mul_(1./255)),
    T.Normalize(
        mean=[-0.485/0.299, -0.456/0.224, -0.406/0.225],
        std=[1/0.229, 1/0.224, 1/0.255])
])

In [ ]:
#@ Gram Matrix module:
class GramMatrix(nn.Module):
  def forward(self, input):
    b, c, h, w=input.size()
    feature=input.view(b, c, h*w)
    G= feature @ feature.transpose(1, 2)
    G.div_(h*w)
    return G

In [ ]:
class GramMSELoss(nn.Module):
  def forward(self, input, target):
    output=F.mse_loss(GramMatrix()(input), target)
    return(out)